In [2]:
import chess
import chess.engine
import chess.pgn
from datetime import datetime
import io
import pandas as pd
from stockfish import Stockfish
import requests



In [4]:
stockfish_path = "/mnt/c/Users/x_BrownBlanC/Coding/stockfish/stockfish-windows-x86-64-sse41-popcnt.exe"
stockfish = Stockfish(path=stockfish_path, depth=20, parameters={"Threads": 10, "Minimum Thinking Time": 30, "Hash": 2048})

In [85]:
chess_username = "CheyDB"
chess_email = "cheydb@rocketmail.com"
url = f"https://api.chess.com/pub/player/{chess_username}/games/archives"
headers = {"User-Agent": f"username:{chess_username}, email:{chess_email}"}

In [82]:
def get_monthly_archive_urls(chess_username=chess_username, url=url, headers=headers):
    response = requests.get(url=url, headers=headers)
    monthly_archive_urls = response.json()["archives"]
    return monthly_archive_urls

In [87]:
monthly_archive_urls = get_monthly_archive_urls()

In [98]:
def get_games_from_monthly_archive(monthly_archive_url):
    response = requests.get(monthly_archive_url, headers=headers)
    games = response.json()["games"]
    return games

In [92]:
print(monthly_archive_urls)
    

['https://api.chess.com/pub/player/cheydb/games/2018/08', 'https://api.chess.com/pub/player/cheydb/games/2018/09', 'https://api.chess.com/pub/player/cheydb/games/2018/10', 'https://api.chess.com/pub/player/cheydb/games/2018/11', 'https://api.chess.com/pub/player/cheydb/games/2018/12', 'https://api.chess.com/pub/player/cheydb/games/2019/01', 'https://api.chess.com/pub/player/cheydb/games/2019/02', 'https://api.chess.com/pub/player/cheydb/games/2019/03', 'https://api.chess.com/pub/player/cheydb/games/2019/04', 'https://api.chess.com/pub/player/cheydb/games/2019/05', 'https://api.chess.com/pub/player/cheydb/games/2019/06', 'https://api.chess.com/pub/player/cheydb/games/2019/07', 'https://api.chess.com/pub/player/cheydb/games/2019/09', 'https://api.chess.com/pub/player/cheydb/games/2019/10', 'https://api.chess.com/pub/player/cheydb/games/2019/11', 'https://api.chess.com/pub/player/cheydb/games/2019/12', 'https://api.chess.com/pub/player/cheydb/games/2020/01', 'https://api.chess.com/pub/pla

In [101]:
games_list
for monthly_archive_url in monthly_archive_urls:
    print(monthly_archive_url)
    games = get_games_from_monthly_archive(monthly_archive_url)
    games_list.extend(games)
print(len(games_list))
    

https://api.chess.com/pub/player/cheydb/games/2018/08
https://api.chess.com/pub/player/cheydb/games/2018/09
https://api.chess.com/pub/player/cheydb/games/2018/10
https://api.chess.com/pub/player/cheydb/games/2018/11
https://api.chess.com/pub/player/cheydb/games/2018/12
https://api.chess.com/pub/player/cheydb/games/2019/01
https://api.chess.com/pub/player/cheydb/games/2019/02
https://api.chess.com/pub/player/cheydb/games/2019/03
https://api.chess.com/pub/player/cheydb/games/2019/04
https://api.chess.com/pub/player/cheydb/games/2019/05
https://api.chess.com/pub/player/cheydb/games/2019/06
https://api.chess.com/pub/player/cheydb/games/2019/07
https://api.chess.com/pub/player/cheydb/games/2019/09
https://api.chess.com/pub/player/cheydb/games/2019/10
https://api.chess.com/pub/player/cheydb/games/2019/11
https://api.chess.com/pub/player/cheydb/games/2019/12
https://api.chess.com/pub/player/cheydb/games/2020/01
https://api.chess.com/pub/player/cheydb/games/2020/02
https://api.chess.com/pub/pl

In [103]:
for game in games_list[:10]:
    print(game)

{'url': 'https://www.chess.com/game/live/3023955132', 'pgn': '[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2018.08.22"]\n[Round "-"]\n[White "CheyDB"]\n[Black "raghav777701"]\n[Result "1-0"]\n[CurrentPosition "1nbqkbnr/1p2p3/2pp1p2/2r4p/N1BPP1pN/pP4B1/P1P2PPP/R2Q1RK1 b k -"]\n[Timezone "UTC"]\n[ECO "B00"]\n[ECOUrl "https://www.chess.com/openings/Kings-Pawn-Opening-Ware-Defense-2.d4"]\n[UTCDate "2018.08.22"]\n[UTCTime "17:56:20"]\n[WhiteElo "1022"]\n[BlackElo "924"]\n[TimeControl "600"]\n[Termination "CheyDB won - game abandoned"]\n[StartTime "17:56:20"]\n[EndDate "2018.08.22"]\n[EndTime "18:04:30"]\n[Link "https://www.chess.com/game/live/3023955132"]\n\n1. e4 {[%clk 0:09:58.1]} 1... a5 {[%clk 0:09:48.5]} 2. d4 {[%clk 0:09:54]} 2... c6 {[%clk 0:09:40.9]} 3. Nc3 {[%clk 0:09:47.5]} 3... d6 {[%clk 0:09:24.9]} 4. Nf3 {[%clk 0:09:42.6]} 4... h5 {[%clk 0:09:14.8]} 5. Bg5 {[%clk 0:09:29.6]} 5... f6 {[%clk 0:09:09.3]} 6. Bh4 {[%clk 0:09:23.7]} 6... g5 {[%clk 0:08:54]} 7. Bg3 {[%clk 0:09:02.

In [79]:
def get_live_games(games):
    live_games = []
    for game in games:
        if game["url"].split("/")[-2] == "live":
            live_games.append(game)
    return live_games

In [105]:
live_games = get_live_games(games_list)
print(len(live_games))
for live_game in live_games[:10]:
    print(live_game)

1041
{'url': 'https://www.chess.com/game/live/3023955132', 'pgn': '[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2018.08.22"]\n[Round "-"]\n[White "CheyDB"]\n[Black "raghav777701"]\n[Result "1-0"]\n[CurrentPosition "1nbqkbnr/1p2p3/2pp1p2/2r4p/N1BPP1pN/pP4B1/P1P2PPP/R2Q1RK1 b k -"]\n[Timezone "UTC"]\n[ECO "B00"]\n[ECOUrl "https://www.chess.com/openings/Kings-Pawn-Opening-Ware-Defense-2.d4"]\n[UTCDate "2018.08.22"]\n[UTCTime "17:56:20"]\n[WhiteElo "1022"]\n[BlackElo "924"]\n[TimeControl "600"]\n[Termination "CheyDB won - game abandoned"]\n[StartTime "17:56:20"]\n[EndDate "2018.08.22"]\n[EndTime "18:04:30"]\n[Link "https://www.chess.com/game/live/3023955132"]\n\n1. e4 {[%clk 0:09:58.1]} 1... a5 {[%clk 0:09:48.5]} 2. d4 {[%clk 0:09:54]} 2... c6 {[%clk 0:09:40.9]} 3. Nc3 {[%clk 0:09:47.5]} 3... d6 {[%clk 0:09:24.9]} 4. Nf3 {[%clk 0:09:42.6]} 4... h5 {[%clk 0:09:14.8]} 5. Bg5 {[%clk 0:09:29.6]} 5... f6 {[%clk 0:09:09.3]} 6. Bh4 {[%clk 0:09:23.7]} 6... g5 {[%clk 0:08:54]} 7. Bg3 {[%clk 0:0

In [80]:
def get_daily_games(games):
    daily_games = []
    for game in games:
        if game["url"].split("/")[-2] == "daily":
            daily_games.append(game)
    return daily_games

In [106]:
daily_games = get_daily_games(games_list)
print(len(daily_games))
for daily_game in daily_games[:10]:
    print(daily_game)

313
{'url': 'https://www.chess.com/game/daily/204755500', 'pgn': '[Event "Let\'s Play!"]\n[Site "Chess.com"]\n[Date "2018.09.18"]\n[Round "-"]\n[White "CheyDB"]\n[Black "ah2000"]\n[Result "0-1"]\n[CurrentPosition "r4rk1/1pp2ppp/8/p4b2/1qP5/1K3P1P/PB1P4/R2Q1B1R w - - 4 22"]\n[Timezone "UTC"]\n[ECO "C30"]\n[ECOUrl "https://www.chess.com/openings/Kings-Gambit-Declined-Queens-Knight-Defense"]\n[UTCDate "2018.09.18"]\n[UTCTime "16:41:08"]\n[WhiteElo "753"]\n[BlackElo "1189"]\n[TimeControl "1/604800"]\n[Termination "ah2000 won by checkmate"]\n[StartTime "16:41:08"]\n[EndDate "2018.09.25"]\n[EndTime "22:48:23"]\n[Link "https://www.chess.com/game/daily/204755500"]\n\n1. e4 e5 2. f4 Nc6 3. b3 Nf6 4. Nc3 d5 5. fxe5 Nxe5 6. Nf3 Bd6 7. Nxd5 Nxd5 8. exd5 Bg4 9. h3 Nxf3+ 10. gxf3 Bg3+ 11. Ke2 Qe7+ 12. Kd3 Bf5+ 13. Kc3 Qc5+ 14. Kb2 Qxd5 15. c4 Qd4+ 16. Ka3 Bd6+ 17. b4 O-O 18. Bb2 Bxb4+ 19. Kxb4 a5+ 20. Ka3 Qc5+ 21. Kb3 Qb4# 0-1\n', 'time_control': '1/604800', 'end_time': 1537915703, 'rated': True, 't

In [117]:
def extract_pgn(games_list):
    pgn_list = []
    for game in games_list:
        pgn_list.append(game["pgn"])
    return pgn_list

In [119]:
pgn_list = extract_pgn(live_games[:3])

In [124]:
str(pgn_list[1])

'[\'[Event "Live Chess"]\\n[Site "Chess.com"]\\n[Date "2018.08.22"]\\n[Round "-"]\\n[White "CheyDB"]\\n[Black "raghav777701"]\\n[Result "1-0"]\\n[CurrentPosition "1nbqkbnr/1p2p3/2pp1p2/2r4p/N1BPP1pN/pP4B1/P1P2PPP/R2Q1RK1 b k -"]\\n[Timezone "UTC"]\\n[ECO "B00"]\\n[ECOUrl "https://www.chess.com/openings/Kings-Pawn-Opening-Ware-Defense-2.d4"]\\n[UTCDate "2018.08.22"]\\n[UTCTime "17:56:20"]\\n[WhiteElo "1022"]\\n[BlackElo "924"]\\n[TimeControl "600"]\\n[Termination "CheyDB won - game abandoned"]\\n[StartTime "17:56:20"]\\n[EndDate "2018.08.22"]\\n[EndTime "18:04:30"]\\n[Link "https://www.chess.com/game/live/3023955132"]\\n\\n1. e4 {[%clk 0:09:58.1]} 1... a5 {[%clk 0:09:48.5]} 2. d4 {[%clk 0:09:54]} 2... c6 {[%clk 0:09:40.9]} 3. Nc3 {[%clk 0:09:47.5]} 3... d6 {[%clk 0:09:24.9]} 4. Nf3 {[%clk 0:09:42.6]} 4... h5 {[%clk 0:09:14.8]} 5. Bg5 {[%clk 0:09:29.6]} 5... f6 {[%clk 0:09:09.3]} 6. Bh4 {[%clk 0:09:23.7]} 6... g5 {[%clk 0:08:54]} 7. Bg3 {[%clk 0:09:02.6]} 7... a4 {[%clk 0:08:36.3]} 8. Bc

In [154]:
# stockfish.make_moves_from_current_position(chess.pgn.read_game(io.StringIO(pgn_list[1])).mainline_moves())
# evaluation = stockfish.get_evaluation()
# print(evaluation)
# stockfish.get_fen_position()
stockfish.get_board_visual()
    

'+---+---+---+---+---+---+---+---+\n| r | n | b | q | k | b | n | r | 8\n+---+---+---+---+---+---+---+---+\n| p | p | p | p | p | p | p | p | 7\n+---+---+---+---+---+---+---+---+\n|   |   |   |   |   |   |   |   | 6\n+---+---+---+---+---+---+---+---+\n|   |   |   |   |   |   |   |   | 5\n+---+---+---+---+---+---+---+---+\n|   |   |   |   | P |   |   |   | 4\n+---+---+---+---+---+---+---+---+\n|   |   |   |   |   |   |   |   | 3\n+---+---+---+---+---+---+---+---+\n| P | P | P | P |   | P | P | P | 2\n+---+---+---+---+---+---+---+---+\n| R | N | B | Q | K | B | N | R | 1\n+---+---+---+---+---+---+---+---+\n  a   b   c   d   e   f   g   h\n'

In [130]:
 Move_score = f"{evaluation['type']} : {evaluation['value']}"

In [131]:
print(Move_score)

cp : -504


In [73]:
data = []
board = chess.Board()
for move_number, move in enumerate(game.mainline_moves(), 1):
    board.push(move)
    move_info = {
        'Game ID': 'Your_Game_ID',  # Assign appropriate ID
        'Move Number': move_number,
        'Move': move.uci(),
        'Move Color': 'White' if move_number % 2 else 'Black',
        'Piece Moved': board.piece_at(move.to_square),
        'Starting Square': chess.SQUARE_NAMES[move.from_square],
        'Ending Square': chess.SQUARE_NAMES[move.to_square],
        'Capture': board.is_capture(move),  # Checks if the move is a capture
        'Check': board.is_check(),
        'Checkmate': board.is_checkmate(),
        'Promotion': bool(move.promotion),
        'Comments': None  # Placeholder for potential annotations or comments
    }
    print(board)
    data.append(move_info)

df_moves = pd.DataFrame(data)
print(df_moves)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
r n b q k b n r
. p p p p p p p
. . . . . . . .
p . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
r n b q k b n r
. p p p p p p p
. . . . . . . .
p . . . . . . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
r n b q k b n r
. p . p p p p p
. . p . . . . .
p . . . . . . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
r n b q k b n r
. p . p p p p p
. . p . . . . .
p . . . . . . .
. . . P P . . .
. . N . . . . .
P P P . . P P P
R . B Q K B N R
r n b q k b n r
. p . . p p p p
. . p p . . . .
p . . . . . . .
. . . P P . . .
. . N . . . . .
P P P . . P P P
R . B Q K B N R
r n b q k b n r
. p . . p p p p
. . p p . . . .
p . . . . . . .
. . . P P . . .
. . N . . N . .
P P P . . P P P
R . B Q K B . R
r n b q k b n r
. p . . p p p .
. . p p . . . .
p . . . . . . p
. . . P P . . .
. . N . . N . .
P P P . 

In [164]:
def parse_pgn(pgn_text):
    pgn = io.StringIO(pgn_text)
    game = chess.pgn.read_game(pgn)

    # Extract game-level data
    game_data = {
        "Game ID": game.headers["Link"].split("/")[-1],
        "Date": game.headers.get("Date"),
        "Start Time": game.headers.get("UTCTime"),
        "End Time": game.headers.get("EndTime"),
        "White Player": game.headers.get("White"),
        "Black Player": game.headers.get("Black"),
        "White Player ELO": game.headers.get("WhiteElo"),
        "Black Player ELO": game.headers.get("BlackElo"),
        "Final Position(FEN)": game.headers.get("CurrentPosition"),
        "Opening Name": game.headers.get("Opening", ""),
        "ECO": game.headers.get("ECO"),
        "ECO url": game.headers.get("ECOUrl"),
        "Time Control": game.headers.get("TimeControl"),
        "Winner": game.headers.get("Result"),
        "I Won": "CheyDB" in [game.headers.get("White"), game.headers.get("Black")] and "CheyDB" == game.headers.get("Winner"),
        "Result": game.headers.get("Result"),
        "Termination": game.headers.get("Termination"),
        "Total Time Spent": str(datetime.strptime(game.headers.get("EndTime"), '%H:%M:%S') - datetime.strptime(game.headers.get("UTCTime"), '%H:%M:%S')),
        "Game Link": game.headers.get("Link")
    }

    # Extract move-level data
    move_data = []
    move_number = 1
    board = chess.Board()
    for move in game.mainline_moves():
        # if move_number > 1:
        fen_before = board.fen()  # Capture FEN before the move
        stockfish_fen_position = stockfish.set_fen_position(fen_before)
        best_move = stockfish.get_best_move()
        board_eval_before_move = stockfish.get_evaluation()
        san = board.san(move)
        board.push(move)
        fen_after = board.fen()
        print(fen_after)
        stockfish_fen_position = stockfish.set_fen_position(fen_after)
        board_eval_after_move = stockfish.get_evaluation()

        move_data.append({
            "Game ID": game.headers["Link"].split("/")[-1],
            "Move Number": (move_number + 1) // 2,  # Adjusting move number to increment after each full cycle (white+black)
            "Move": move.uci(),
            "San": san,
            "Move Colour": "White" if board.turn == chess.BLACK else "Black",  # Color is based on turn after move
            "Piece Moved": board.piece_at(move.to_square),
            "Starting Square": chess.square_name(move.from_square),
            "Ending Square": chess.square_name(move.to_square),
            "FEN Before Move": fen_before,
            "FEN After Move": fen_after,
            "Best Move": best_move,
            "Move Score": None,
            "Board Eval Before Move(type)": board_eval_before_move['type'],
            "Board Eval Before Move(value)": board_eval_before_move['value'],
            "Board Eval After Move(type)": board_eval_after_move['type'],
            "Board Eval After Move(type)": board_eval_after_move['value'],
            "Time Taken": None,
            "Capture": board.is_capture(move),
            "Check": board.is_check(),
            "Checkmate": board.is_checkmate(),
            "Promotion": bool(move.promotion)
        })
        
        # Update move number
        if board.turn == chess.WHITE:
            move_number += 1

    return game_data, move_data

In [165]:
for pgn in pgn_list[:2]:
    game_data, move_data = parse_pgn(pgn)
    game_data_df = pd.DataFrame(game_data)
    move_data_df = pd.DataFrame(game_data)

rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1
rnbqkbnr/1ppppppp/8/p7/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
rnbqkbnr/1ppppppp/8/p7/3PP3/8/PPP2PPP/RNBQKBNR b KQkq - 0 2
rnbqkbnr/1p1ppppp/2p5/p7/3PP3/8/PPP2PPP/RNBQKBNR w KQkq - 0 3
rnbqkbnr/1p1ppppp/2p5/p7/3PP3/2N5/PPP2PPP/R1BQKBNR b KQkq - 1 3
rnbqkbnr/1p2pppp/2pp4/p7/3PP3/2N5/PPP2PPP/R1BQKBNR w KQkq - 0 4
rnbqkbnr/1p2pppp/2pp4/p7/3PP3/2N2N2/PPP2PPP/R1BQKB1R b KQkq - 1 4
rnbqkbnr/1p2ppp1/2pp4/p6p/3PP3/2N2N2/PPP2PPP/R1BQKB1R w KQkq - 0 5
rnbqkbnr/1p2ppp1/2pp4/p5Bp/3PP3/2N2N2/PPP2PPP/R2QKB1R b KQkq - 1 5
rnbqkbnr/1p2p1p1/2pp1p2/p5Bp/3PP3/2N2N2/PPP2PPP/R2QKB1R w KQkq - 0 6
rnbqkbnr/1p2p1p1/2pp1p2/p6p/3PP2B/2N2N2/PPP2PPP/R2QKB1R b KQkq - 1 6
rnbqkbnr/1p2p3/2pp1p2/p5pp/3PP2B/2N2N2/PPP2PPP/R2QKB1R w KQkq - 0 7
rnbqkbnr/1p2p3/2pp1p2/p5pp/3PP3/2N2NB1/PPP2PPP/R2QKB1R b KQkq - 1 7
rnbqkbnr/1p2p3/2pp1p2/6pp/p2PP3/2N2NB1/PPP2PPP/R2QKB1R w KQkq - 0 8
rnbqkbnr/1p2p3/2pp1p2/6pp/p1BPP3/2N2NB1/PPP2PPP/R2QK2R b KQkq - 1 8
rnbqkbnr/1p2p3/2pp1p2

ValueError: If using all scalar values, you must pass an index

In [ ]:
game_data_df.head()
move_data_df.head()

In [166]:
game_data

{'Game ID': '3023955132',
 'Date': '2018.08.22',
 'Start Time': '17:56:20',
 'End Time': '18:04:30',
 'White Player': 'CheyDB',
 'Black Player': 'raghav777701',
 'White Player ELO': '1022',
 'Black Player ELO': '924',
 'Final Position(FEN)': '1nbqkbnr/1p2p3/2pp1p2/2r4p/N1BPP1pN/pP4B1/P1P2PPP/R2Q1RK1 b k -',
 'Opening Name': '',
 'ECO': 'B00',
 'ECO url': 'https://www.chess.com/openings/Kings-Pawn-Opening-Ware-Defense-2.d4',
 'Time Control': '600',
 'Winner': '1-0',
 'I Won': False,
 'Result': '1-0',
 'Termination': 'CheyDB won - game abandoned',
 'Total Time Spent': '0:08:10',
 'Game Link': 'https://www.chess.com/game/live/3023955132'}

In [167]:
game_data_df = pd.DataFrame(game_data, index=[0])

In [169]:
move_data

[{'Game ID': '3023955132',
  'Move Number': 1,
  'Move': 'e2e4',
  'San': 'e4',
  'Move Colour': 'White',
  'Piece Moved': Piece.from_symbol('P'),
  'Starting Square': 'e2',
  'Ending Square': 'e4',
  'FEN Before Move': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
  'FEN After Move': 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1',
  'Best Move': 'e2e4',
  'Move Score': None,
  'Board Eval Before Move(type)': 'cp',
  'Board Eval Before Move(value)': 35,
  'Board Eval After Move(type)': 35,
  'Time Taken': None,
  'Capture': True,
  'Check': False,
  'Checkmate': False,
  'Promotion': False},
 {'Game ID': '3023955132',
  'Move Number': 1,
  'Move': 'a7a5',
  'San': 'a5',
  'Move Colour': 'Black',
  'Piece Moved': Piece.from_symbol('p'),
  'Starting Square': 'a7',
  'Ending Square': 'a5',
  'FEN Before Move': 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1',
  'FEN After Move': 'rnbqkbnr/1ppppppp/8/p7/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2',
  'Best 

In [170]:
move_data_df = pd.DataFrame(move_data)

In [173]:
move_data_df

,Game ID,Move Number,Move,San,Move Colour,Piece Moved,Starting Square,Ending Square,FEN Before Move,FEN After Move,Best Move,Move Score,Board Eval Before Move(type),Board Eval Before Move(value),Board Eval After Move(type),Time Taken,Capture,Check,Checkmate,Promotion
0,3023955132,1,e2e4,e4,White,P,e2,e4,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,e2e4,None,cp,35,35,None,True,False,False,False
1,3023955132,1,a7a5,a5,Black,p,a7,a5,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,rnbqkbnr/1ppppppp/8/p7/4P3/8/PPPP1PPP/RNBQKBNR...,e7e5,None,cp,22,90,None,True,False,False,False
2,3023955132,1,d2d4,d4,White,P,d2,d4,rnbqkbnr/1ppppppp/8/p7/4P3/8/PPPP1PPP/RNBQKBNR...,rnbqkbnr/1ppppppp/8/p7/3PP3/8/PPP2PPP/RNBQKBNR...,d2d4,None,cp,98,97,None,True,False,False,False
3,3023955132,1,c7c6,c6,Black,p,c7,c6,rnbqkbnr/1ppppppp/8/p7/3PP3/8/PPP2PPP/RNBQKBNR...,rnbqkbnr/1p1ppppp/2p5/p7/3PP3/8/PPP2PPP/RNBQKB...,g7g6,None,cp,99,97,None,True,False,False,False
4,3023955132,2,b1c3,Nc3,White,N,b1,c3,rnbqkbnr/1p1ppppp/2p5/p7/3PP3/8/PPP2PPP/RNBQKB...,rnbqkbnr/1p1ppppp/2p5/p7/3PP3/2N5/PPP2PPP/R1BQ...,b1c3,None,cp,94,85,None,True,False,False,False
5,3023955132,2,d7d6,d6,Black,p,d7,d6,rnbqkbnr/1p1ppppp/2p5/p7/3PP3/2N5/PPP2PPP/R1BQ...,rnbqkbnr/1p2pppp/2pp4/p7/3PP3/2N5/PPP2PPP/R1BQ...,d7d6,None,cp,92,93,None,True,False,False,False
6,3023955132,2,g1f3,Nf3,White,N,g1,f3,rnbqkbnr/1p2pppp/2pp4/p7/3PP3/2N5/PPP2PPP/R1BQ...,rnbqkbnr/1p2pppp/2pp4/p7/3PP3/2N2N2/PPP2PPP/R1...,g1f3,None,cp,91,98,None,True,False,False,False
7,3023955132,2,h7h5,h5,Black,p,h7,h5,rnbqkbnr/1p2pppp/2pp4/p7/3PP3/2N2N2/PPP2PPP/R1...,rnbqkbnr/1p2ppp1/2pp4/p6p/3PP3/2N2N2/PPP2PPP/R...,g8f6,None,cp,95,166,None,True,False,False,False
8,3023955132,3,c1g5,Bg5,White,B,c1,g5,rnbqkbnr/1p2ppp1/2pp4/p6p/3PP3/2N2N2/PPP2PPP/R...,rnbqkbnr/1p2ppp1/2pp4/p5Bp/3PP3/2N2N2/PPP2PPP/...,c1e3,None,cp,169,154,None,True,False,False,False
9,3023955132,3,f7f6,f6,Black,p,f7,f6,rnbqkbnr/1p2ppp1/2pp4/p5Bp/3PP3/2N2N2/PPP2PPP/...,rnbqkbnr/1p2p1p1/2pp1p2/p5Bp/3PP3/2N2N2/PPP2PP...,g7g6,None,cp,160,215,None,True,False,False,False


In [174]:
board.is_capture(move)

False

In [175]:
print(move)

e2e3


In [1]:
game_data_df

NameError: name 'game_data_df' is not defined